### 3.3. Simple Self-Attention Mechanism without trainable weights

In [3]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
) # (input_size, output_dim)

#### Single Context Vector
1. Get **Attention Score**: How similar between query and input (relevant)

In [5]:
# 1. Get Attention Score of each token with query: dot product (element-wise)

input_query = inputs[1] # x_2

attn_scores_2 = torch.empty(inputs.shape[0]) # input_size = how many tokens
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, input_query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


2. Compute **Attention Weight** - Trainable weight

In [12]:
# 2. Attention Weight: Normalization

# 2-1) normalization using sum
attn_weights2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention Weights:", attn_weights2_2_tmp)
print("Sum:", attn_weights2_2_tmp.sum())

# 2-2) softmax
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
attn_weights2 = softmax_naive(attn_scores_2)
print("Softmax:", attn_weights2_2)
print("Sum of Softmax:", attn_weights2_2.sum())

Attention Weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)
Softmax: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum of Softmax: tensor(1.)


In [13]:
torch.softmax(attn_scores_2, dim=0)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

3. Compute **Context Vector**(z_i): attention weight * input token

In [16]:
# 3. Context Vector (shape == query.shape)
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    print(f"Attn_weight:{attn_weights2[i]} ---> Input:{inputs[i]}")
    context_vec_2 += attn_weights2[i] * x_i
print(context_vec_2)

Attn_weight:0.13854758441448212 ---> Input:tensor([0.4300, 0.1500, 0.8900])
Attn_weight:0.2378913313150406 ---> Input:tensor([0.5500, 0.8700, 0.6600])
Attn_weight:0.23327405750751495 ---> Input:tensor([0.5700, 0.8500, 0.6400])
Attn_weight:0.12399159371852875 ---> Input:tensor([0.2200, 0.5800, 0.3300])
Attn_weight:0.10818187892436981 ---> Input:tensor([0.7700, 0.2500, 0.1000])
Attn_weight:0.15811361372470856 ---> Input:tensor([0.0500, 0.8000, 0.5500])
tensor([0.4419, 0.6515, 0.5683])


#### Computing Attention weights for all input tokens
- context vector.shape == input_embedding.shape

In [18]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [20]:
# 1) get attention score: shape --> (input_num, input_num)

# Approach 1) multiplication
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

# Approach 2) matrix multiplication
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [36]:
# 2) Attention weight: Normalization (softmax)
# dim=-1: make the columns sum as -1
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [22]:
# 3) Context vector: x_i * attn_weights
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [37]:
## [summary] all process (input: input_num x output_dim)
attn_scores = inputs @ inputs.T # input_num x input_num
attn_weights = torch.softmax(attn_scores, dim=-1) # input_num x input_num
all_context_vecs = attn_weights @ inputs # input_num x output_dim

### 3.4. Self-attention with trainable weights

In [24]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

#### Single self-attention

In [26]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2 # output_embedding size: flexible

In [31]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out)) # make trainable matrices
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

In [32]:
query_2 = x_2 @ W_query # (input_num, em_in) * (em_in, em_out)
query_2 # (input_num, em_out)

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [33]:
keys = inputs @ W_key
value = inputs @ W_value

print(keys)

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)


In [34]:
keys_2 = keys[1]
attn_scores_22 = torch.dot(query_2, keys_2)
print(attn_scores_22)

tensor(1.8524, grad_fn=<DotBackward0>)


In [35]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)

In [39]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print("2nd Attention weight:", attn_weights_2)
print("Sum", attn_weights_2.sum())

2nd Attention weight: tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)
Sum tensor(1.0000, grad_fn=<SumBackward0>)


In [41]:
context_vec_2 = attn_weights_2 @ value
context_vec_2

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)

#### Implementing compact SelfAttention class

In [43]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        # 1) attention score
        attn_scores = queries @ keys.T # (d_in, d_in)
        # 2) attention weight
        attn_weights = torch.softmax(
            attn_scores/keys.shape[-1] ** 0.5, dim=-1
        ) # (d_in, d_in)
        # 3) context_vector
        context_vector = attn_weights @ values # (d_in, d_out)
        return context_vector

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [44]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, bias=False): # don't need to use bias
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=bias)
        self.W_key = nn.Linear(d_in, d_out, bias=bias)
        self.W_value = nn.Linear(d_in, d_out, bias=bias)
    
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # 1) attention score
        attn_scores = queries @ keys.T # (d_in, d_in)
        # 2) attention weight
        attn_weights = torch.softmax(
            attn_scores/keys.shape[-1] ** 0.5, dim=-1
        ) # (d_in, d_in)
        # 3) context_vector
        context_vector = attn_weights @ values # (d_in, d_out)
        return context_vector

torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


### 3.5 Causal Attention (Hiding future words)

#### Diagonal Masking (1)
- zero masking
    - get attention weight
    - diagonal masking
    - normalized

In [45]:
# ex) Your journey starts with one step
# Your --> journey

queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
values = sa_v2.W_value(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores/values.shape[-1]**0.5, dim=-1)

In [46]:
attn_weights

tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)

In [47]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [48]:
# get masked attention weight
masked_simple = attn_weights * mask_simple
masked_simple

tensor([[0.1717, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1749, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1637, 0.1749, 0.1746, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.0000, 0.0000],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<MulBackward0>)

In [49]:
# normalize the masked values
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<DivBackward0>)


#### Diagonal Masking (1)
- negative inf masking
    - get attention score
    - masked with negative infinite values
    - apply softmax

In [51]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)


In [52]:
attn_weights = torch.softmax(masked/keys.shape[-1]**0.5, dim=-1)
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)

#### Masking additional attention weights with dropout
- not using actively now

In [54]:
torch.manual_seed(123)
layer = torch.nn.Dropout(0.5)
example = torch.ones(inputs.shape[0], inputs.shape[0])

print(layer(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [55]:
print(layer(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6816, 0.6804, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5085, 0.4936, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3906, 0.0000],
        [0.3249, 0.3418, 0.0000, 0.3308, 0.3249, 0.3363]],
       grad_fn=<MulBackward0>)


In [56]:
dropout_rate = 0.5
1 / (1-dropout_rate)

2.0

#### Implementing a compact causal self-attention class

In [66]:
batches = torch.stack((inputs, inputs), dim=0)
print(batches.shape)

torch.Size([2, 6, 3])


In [68]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, bias=False):
        super().__init__()
        
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=bias)
        self.W_key = nn.Linear(d_in, d_out, bias=bias)
        self.W_value = nn.Linear(d_in, d_out, bias=bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        batch, num_tokens, d_in = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1,2) # [batch, num_tokens, num_tokens]
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )
        attn_weights = torch.softmax(
            attn_scores/values.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values # [batch, num_tokens, d_out]
        return context_vec

torch.manual_seed(123)
context_length = batches.shape[1]
dropout = 0.0
ca = CausalAttention(d_in, d_out, context_length, dropout)
context_vecs = ca(batches)
print(context_vecs)
print("Shape:", context_vecs.shape)

tensor([[[-0.3132, -0.2272,  0.4772,  0.1063],
         [-0.2320,  0.0293,  0.5789,  0.3056],
         [-0.2068,  0.1162,  0.6118,  0.3695],
         [-0.1635,  0.1328,  0.5457,  0.3531],
         [-0.1687,  0.1813,  0.5315,  0.3400],
         [-0.1411,  0.1727,  0.5063,  0.3432]],

        [[-0.3132, -0.2272,  0.4772,  0.1063],
         [-0.2320,  0.0293,  0.5789,  0.3056],
         [-0.2068,  0.1162,  0.6118,  0.3695],
         [-0.1635,  0.1328,  0.5457,  0.3531],
         [-0.1687,  0.1813,  0.5315,  0.3400],
         [-0.1411,  0.1727,  0.5063,  0.3432]]], grad_fn=<UnsafeViewBackward0>)
Shape: torch.Size([2, 6, 4])


### 3.6 Multi-head attention

In [70]:
## 1) stacking causal attention

class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads=2):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout) for _ in range(num_heads)]
        )
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

torch.manual_seed(123)

context_length = batches.shape[1]
d_in, d_out = batches.shape[-1], 2
dropout = 0.0
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, dropout, num_heads=2
)
context_vecs = mha(batches)
print(context_vecs)
print("Shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
Shape: torch.Size([2, 6, 4])


In [87]:
## 2) multi-head attention with weight splits
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out # output dimension
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        
        self.W_query = nn.Linear(d_in, d_out, bias=bias)
        self.W_key = nn.Linear(d_in, d_out, bias=bias)
        self.W_value = nn.Linear(d_in, d_out, bias=bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        self.proj = nn.Linear(d_out, d_out)

    def forward(self, x):
        batch, num_tokens, d_in = x.shape

        keys = self.W_key(x) # [batch, num_tokens, d_out]
        queries = self.W_query(x) # [batch, num_tokens, d_out]
        values = self.W_value(x) # [batch, num_tokens, d_out]

        # devide into head => [batch, num_tokens, num_heads, head_dim] d_out = num_head * head_dim
        keys = keys.view(batch, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(batch, num_tokens, self.num_heads, self.head_dim)
        values = values.view(batch, num_tokens, self.num_heads, self.head_dim)
        
        # 1) get attention score (Q @ K.T) ---> [ batch, num_head, num_tokens, num_tokens ]
        # 1-1) want to calculate attention for each head
        keys = keys.transpose(1,2) # [batch, num_heads, num_tokens, head_dim]
        queries = queries.transpose(1,2) # [batch, num_heads, num_tokens, head_dim]
        values = values.transpose(1,2) # [batch, num_heads, num_tokens, head_dim]

        attn_scores = queries @ keys.transpose(2, 3) # [batch, num_head, num_tokens, num_tokens]
        # [batch, num_head, num_tokens, head_dim] @ [batch, num_head, head_dim, num_tokens]

        ## 1-2) masking
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        # 2) attention weight: normalization
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1) # [batch, num_head, num_tokens, num_tokens]
        attn_weights = self.dropout(attn_weights)

        # 3) get context vector ---> [ batch, num_tokens, d_out(num_head * head_dim) ]
        # attn_weights @ values ---> [batch, num_head, num_tokens, head_dim]
        # (attn_weights @ values).transpose(1,2) ---> [batch, num_tokens, num_head, head_dim]
        context_vec = (attn_weights @ values).transpose(1,2)

        ## 3-2) reshape to have original shape
        context_vec = context_vec.reshape(batch, num_tokens, self.d_out)
        context_vec = self.proj(context_vec)
        
        return context_vec # [batch, num_tokens, d_out]

torch.manual_seed(123)

batch, context_length, d_in = batches.shape
d_out = 4
dropout = 0.0
mha = MultiHeadAttention(
    d_in, d_out, context_length, dropout, num_heads=2
)
context_vecs = mha(batches)
print(context_vecs)
print("Shape:", context_vecs.shape)

tensor([[[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]],

        [[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]]], grad_fn=<ViewBackward0>)
Shape: torch.Size([2, 6, 4])


#### FlashAttention
- scaled_dot_product_attention

In [98]:
class MHA_FlashAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads, dropout=0.0, bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "embed_dim is indivisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.context_length = context_length

        self.qkv = nn.Linear(d_in, 3*d_out, bias=bias)
        self.proj = nn.Linear(d_out, d_out)
        self.dropout = dropout

    def forward(self, x):
        batch, num_tokens, in_dim = x.shape

        # apply qkv ---> [batch, num_tokens, 3*out_dim]
        qkv = self.qkv(x)

        # split qkv [batch, num_tokens, 3*out_dim] ---> [batch, num_tokens, 3, num_heads, head_dim]
        qkv = qkv.view(batch, num_tokens, 3, self.num_heads, self.head_dim)
        # [batch, num_tokens, 3, num_heads, head_dim] --> [3, batch, num_head, num_tokens, head_dim]
        qkv = qkv.permute(2, 0, 3, 1, 4)
        queries, keys, values = qkv

        use_dropout = 0. if not self.training else self.dropout

        context_vec = nn.functional.scaled_dot_product_attention(
            queries, keys, values, attn_mask=None, dropout_p=use_dropout, is_causal=True
        ) # [batch, num_head, num_tokens, head_dim]
        context_vec = context_vec.transpose(1,2) # [batch, num_tokens, num_head, head_dim]
        context_vec = context_vec.reshape(batch, num_tokens, self.d_out)
        context_vec = self.proj(context_vec)
        return context_vec # [batch, num_tokens, out_dim]

In [99]:
batch, context_length, d_in = batches.shape
d_out = 4

mha_flashed=MHA_FlashAttention(
    d_in, d_out, context_length, num_heads=2
)
context_vecs = mha_flashed(batches)
print(context_vecs.shape)

torch.Size([2, 6, 4])
